<a href="https://colab.research.google.com/github/skywalker290/Financial-News-Analyser/blob/main/news_article_sentiment_analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1


In [ ]:
!pip install newspaper3k ctransformers langchain --upgrade lxml_html_clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import requests

from bs4 import BeautifulSoup

from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
def get_cnbc_headers(url: str) -> list:
  response = requests.get(url)

  if response.status_code == 200:
      soup = BeautifulSoup(response.content, "html.parser")

      # find classes with news headers
      content_element_news = soup.find_all(class_="RiverHeadline-headline RiverHeadline-hasThumbnail")
      content_element_latest = soup.find_all(class_="LatestNews-headline")

      content_news = []
      content_latest = []

      # extract text from content elements
      for element in content_element_news:
          text = element.get_text(strip=True)
          content_news.append(text)

      for element in content_element_latest:
          text = element.get_text(strip=True)
          content_latest.append(text)

      # combine latest news and news in the main column on the page
      content = content_news + content_latest
      return content

  else:
      print("Failed to retrieve the webpage. Status code:", response.status_code)
      return []

def take_headers_yahoo(url: str) -> list:

  response = requests.get(url)
  if response.status_code == 200:
      soup = BeautifulSoup(response.content, "html.parser")

      links = soup.find_all("a")
      headlines = []



      for link in links:
          link_url = link.get("href")
          if link_url and link_url.startswith(url + "news/"):
              link_response = requests.get(link_url)
              if link_response.status_code == 200:
                  link_soup = BeautifulSoup(link_response.content, "html.parser")
                  headline_elements = link_soup.find_all(attrs={"data-test-locator": "headline"})
                  for headline_element in headline_elements:
                      headline_text = headline_element.text.strip()
                      headlines.append(headline_text)

      return headlines
  else:
      print("Failed to retrieve the webpage. Status code:", response.status_code)
      return []


def get_yahoo_headers(url: str) -> list:

  header=[]
  response = requests.get(url)

  if response.status_code == 200:
      soup = BeautifulSoup(response.content, "html.parser")

      target_tags = soup.find_all("u", class_="StretchedBox")

      for target_tag in target_tags:
          text_after_u = target_tag.find_next_sibling(string=True)

          if text_after_u:
              header.append(text_after_u.strip())
          else:
              print("Failed to find the text")
      return header
  else:
      print("Failed to retrieve the webpage. Status code:", response.status_code)
      return []

In [ ]:
url_yahoo = "https://finance.yahoo.com/"
url_cnbs = "https://www.cnbc.com/world/?region=world"
headers_yahoo = take_headers_yahoo(url_yahoo)
headers_news_yahoo = get_yahoo_headers(url_yahoo)

headers_news_cnbc = get_cnbc_headers(url_cnbs)

df=pd.DataFrame({"News":headers_yahoo+headers_news_yahoo+headers_news_cnbc})

In [ ]:
# # Python 3
# import http.client, urllib.parse

# conn = http.client.HTTPSConnection('api.marketaux.com')

# params = urllib.parse.urlencode({
#     'api_token': 'Wmhg8iGZl1fzEabY7gH8grZsBudBh6CZ0bDqT5gL',
#     'symbols': 'AAPL,TSLA',
#     'limit': 50,
#     })

# conn.request('GET', '/v1/news/all?{}'.format(params))

# res = conn.getresponse()
# data = res.read()

# print(data.decode('utf-8'))

{"warnings":["limit is higher than your plan allows"],"meta":{"found":151384,"returned":3,"limit":3,"page":1},"data":[{"uuid":"456632c7-7673-4307-b125-665491fd21a2","title":"Wall Street Breakfast Podcast: Netflix Dips Despite Beat","description":"Oil gives back gains after latest Mideast worries. Netflix dips despite beating subscriber, financial forecasts. Apple ordered to pull Meta's WhatsApp, Threads from China App Store.","keywords":"","snippet":"MTStock Studio\/E+ via Getty Images\n\nListen below or on the go on Apple Podcasts and Spotify\n\nOil gives back gains after latest Mideast worries. (00:27) Netflix ...","url":"https:\/\/seekingalpha.com\/article\/4684687-wall-street-breakfast-podcast-netflix-dips-plans-to-stop-reporting-subscribers","image_url":"https:\/\/static.seekingalpha.com\/cdn\/s3\/uploads\/getty_images\/1492295886\/image_1492295886.jpg?io=getty-c-w1536","language":"en","published_at":"2024-04-19T11:01:28.000000Z","source":"seekingalpha.com","relevance_score":null,

In [ ]:
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=a952a615523b509a8fb4abdf5c2c1d09e78693dd17216ee60977ac373d5ace42
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=1a7494bad093458c512f29df86c7c8f520e963d64b96af143ef85b4a592f6143
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

In [ ]:
from newspaper import Article
import lxml.html.clean
import nltk
nltk.download('punkt')

#A new article from TOI
url = "https://www.cnbc.com/2024/04/01/google-to-destroy-browsing-data-to-settle-consumer-privacy-lawsuit.html?&qsearchterm=google"

#For different language newspaper refer above table
toi_article = Article(url, language="en") # en for English

#To download the article
toi_article.download()

#To parse the article
toi_article.parse()

#To perform natural language processing ie..nlp
toi_article.nlp()

#To extract title
print("Article's Title:")
print(toi_article.title)
print("n")

#To extract text
print("Article's Text:")
print(toi_article.text)
print("n")

#To extract summary
print("Article's Summary:")
print(toi_article.summary)
print("n")

#To extract keywords
print("Article's Keywords:")
print(toi_article.keywords)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Article's Title:
Google to destroy browsing data to settle consumer privacy lawsuit
n
Article's Text:
A man walks through Google offices on January 25, 2023 in New York City.

Google agreed to destroy billions of data records to settle a lawsuit claiming it secretly tracked the internet use of people who thought they were browsing privately.

Terms of the settlement were filed on Monday in the Oakland, California federal court, and require approval by U.S. District Judge Yvonne Gonzalez Rogers.

Lawyers for the plaintiffs valued the accord at more than $5 billion, and as high as $7.8 billion. Though users will not receive damages, they may still sue individually for damages.

The class action began in 2020, covering millions of Google users who used private browsing since June 1, 2016.

Users alleged that Google's analytics, cookies and apps let the Alphabet unit improperly track people who set Google's Chrome browser to "Incognito" mode and other browsers to "private" browsing mode.
n

In [ ]:
num_words = 10
config={"max_new_tokens":100,"context_length":4096}
llm2 = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",device = "cuda:0",model_file="llama-2-7b-chat.ggmlv3.q5_K_M.bin",callbacks=[StreamingStdOutCallbackHandler()],config=config)


template = """
[INST] <<sys>>
You are a respectful, clear and honest assistent. Your answer is always one sentence. You will be attentive to details. summarise this aritcle with respect to this {Company_name}in less than 250 worlds
<</sys>>
{text}
[/INST]
"""



NameError: name 'CTransformers' is not defined

In [ ]:
prompt = PromptTemplate(template= template,input_variables=["text"])
# news=".".join((df["News"].to_list())[:10])

query = f"""
Here are some news headlines:
{toi_article.title + toi_article.text}

Summarize the market from this news data without highlights in {num_words} or less words.
"""
chain = LLMChain(prompt=prompt, llm=llm2)

response = chain.invoke(query)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, BloomForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = BloomForSequenceClassification.from_pretrained("bigscience/bloom-560m", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BloomForSequenceClassification.from_pretrained(
    "bigscience/bloom-560m", num_labels=num_labels, problem_type="multi_label_classification"
)

labels = torch.sum(
    torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
).to(torch.float)
loss = model(**inputs, labels=labels).loss

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
predicted_class_ids

tensor([0, 1])

In [ ]:
last_hidden_states

tensor([[[ 1.9988e+00,  2.1394e+00, -3.3615e+00,  ..., -1.0710e+03,
          -2.3329e+00,  6.6761e+00],
         [-2.7488e+00, -1.3973e+00, -3.2516e+00,  ..., -9.0800e+02,
          -1.8817e-01,  2.5218e+00],
         [ 5.5479e-01, -2.2881e+00, -4.1895e-01,  ..., -9.4320e+02,
           2.3407e+00,  5.4891e+00],
         [ 1.3067e+00, -3.3615e+00, -1.9271e+00,  ..., -1.0148e+03,
           3.0082e+00,  6.0203e+00],
         [-2.3621e+00, -6.1650e+00, -1.9696e+00,  ..., -9.6746e+02,
           2.2044e+00,  4.4250e+00],
         [ 1.9945e+00, -6.8697e+00,  8.5639e-01,  ..., -9.9383e+02,
           1.5745e+00,  8.3419e+00]]], grad_fn=<NativeLayerNormBackward0>)

In [ ]:
string = "Google terminated 28 employees after a series of protests against labor conditions and the company's contract with the Israeli government to provide cloud computing services. The protesters were arrested, including some who had been locked out of their work accounts and offices. The company stated that it would continue to investigate and take action as needed."

In [ ]:
from transformers import pipeline, BertTokenizer

In [ ]:
# def sliding_window(text, tokenizer, window_size, stride):
#  tokens = tokenizer.tokenize(text)
#  windowed_tokens = [tokens[i:i + window_size] for i in range(0, len(tokens) - window_size + 1, stride)]
#  return windowed_tokens

classifier = pipeline("sentiment-analysis",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
prediction = classifier(" google got slammed!")
print(prediction)

[[{'label': 'sadness', 'score': 0.0008669279050081968}, {'label': 'joy', 'score': 0.0007492421427741647}, {'label': 'love', 'score': 0.00022944044030737132}, {'label': 'anger', 'score': 0.9939999580383301}, {'label': 'fear', 'score': 0.0037806080654263496}, {'label': 'surprise', 'score': 0.0003738906525541097}]]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
prediction

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
# analyzer = SentimentIntensityAnalyzer()
# output = [string].apply(analyzer.polarity_scores).tolist()
scores = sid.polarity_scores(string)
scores

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.117, 'neu': 0.883, 'pos': 0.0, 'compound': -0.7096}

In [ ]:
scores2 = sid.polarity_scores(toi_article.title+toi_article.text)

In [ ]:
scores2

{'neg': 0.053, 'neu': 0.869, 'pos': 0.077, 'compound': 0.461}

# Phase 2

In [ ]:
!pip install newspaper3k

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

from newspaper import Article
import lxml.html.clean
import nltk
nltk.download('punkt')

import spacy


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
links = pd.read_csv('https://raw.githubusercontent.com/skywalker290/Financial-News-Analyser/main/urls.csv')


In [ ]:


def get_article(url):

#A new article from TOI
# url = "https://www.cnbc.com/2024/04/01/google-to-destroy-browsing-data-to-settle-consumer-privacy-lawsuit.html?&qsearchterm=google"
  try:

    article = Article(url, language="en")
    article.download()
    article.parse()
    article.nlp()

    return [article.title, article.text, article.publish_date,article.summary, article.keywords]

  except Exception as e:
    print(url,':',e)
    return []



In [ ]:
def get_articles_data(urls, df):
    data = []
    for url in urls:
        article_data = get_article(url)[:4]
        if article_data:
            data.append(article_data)
    new_df = pd.DataFrame(data, columns=['Title', 'Text', 'PublishDate', 'Summary'])
    df = pd.concat([df, new_df], axis=1)  # Concatenate along columns
    return df

In [ ]:
temp_df = links
tdf = get_articles_data(links['links'],temp_df)

In [ ]:
tdf = tdf.sort_values(by=['PublishDate'], ascending=False)


In [ ]:
def is_related(title, company_name):
    nlp = spacy.load("en_core_web_sm")
    combined_text = title
    doc = nlp(combined_text)
    print(doc)
    for ent in doc.ents:
        if ent.text.lower() == company_name.lower():
            return True
    return False

In [ ]:
# prompt: drop the rows for whcih titile column does not reuturn true

tdf = tdf[tdf['Title'].apply(lambda x: is_related(x, 'Google'))]


A lawyer, a banker and a speech therapist: Meet the 12 jury members of Trump’s hush money trial
Stocks making the biggest moves premarket: Google, HashiCorp, Nvidia and more
GM hires ex-Tesla, Google exec to replace retiring head of manufacturing
Ex-Google software engineer charged with stealing AI technology while working with Chinese companies
Anthropic, backed by Amazon and Google, debuts its most powerful chatbot yet
Google Gemini product lead retreats from social media after troubled AI product launch led to harassment
2 of the top 10 must-see travel experiences in the world are in the U.S., Google data says
Google hit with $2.3 billion lawsuit by Axel Springer, other media groups
Google CEO tells employees Gemini AI blunder ‘unacceptable’
Google expects its more advanced large language models to arrive on Android phones next year
Highmark Health says it's combining tech from Google and Epic to give doctors easier access to information
Google to relaunch Gemini AI picture generato

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
from transformers import BertTokenizer, BertModel
import torch

def preprocess_text(text):
    # Your preprocessing steps here (e.g., removing punctuation, stopwords, etc.)
    return text

def load_word2vec_model():
    # Load pre-trained Word2Vec model
    word2vec_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/data_files/GoogleNews-vectors-negative300.bin', binary=True)
    return word2vec_model

def extract_bow_features(df):
    # Initialize CountVectorizer for Bag-of-Words
    count_vectorizer = CountVectorizer()

    # Combine title and text into a single string
    combined_text = df['Title'] + " " + df['Text']

    # Fit CountVectorizer on combined text
    bow_matrix = count_vectorizer.fit_transform(combined_text)

    return bow_matrix.toarray()

def extract_tfidf_features(df):
    # Initialize TfidfVectorizer for TF-IDF
    tfidf_vectorizer = TfidfVectorizer()

    # Combine title and text into a single string
    combined_text = df['Title'] + " " + df['Text']

    # Fit TfidfVectorizer on combined text
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_text)

    return tfidf_matrix.toarray()

def extract_word_embeddings(df, word2vec_model):
    # Initialize an empty list to store embeddings for each article
    embeddings = []

    # Iterate through each article
    for index, row in df.iterrows():
        title_tokens = row['Title'].split()
        text_tokens = row['Text'].split()

        # Get embeddings for title and text separately
        title_embedding = [word2vec_model[word] for word in title_tokens if word in word2vec_model]
        text_embedding = [word2vec_model[word] for word in text_tokens if word in word2vec_model]

        # Combine embeddings for title and text
        combined_embedding = title_embedding + text_embedding
        embeddings.append(combined_embedding)

    return embeddings

def extract_bert_embeddings(df):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Iterate through each article
    embeddings = []
    for index, row in df.iterrows():
        title = row['Title']
        text = row['Text']

        # Tokenize title and text
        inputs = tokenizer(title, text, return_tensors='pt', max_length=512, truncation=True)

        # Forward pass through BERT model
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract embeddings from BERT output
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())

    return embeddings

def apply_all_techniques(df):
    # Preprocess text
    df['Text'] = df['Text'].apply(preprocess_text)

    # Apply Bag-of-Words
    bow_matrix = extract_bow_features(df)
    df['Bow_features'] = bow_matrix.tolist()

    # # Apply TF-IDF
    # tfidf_matrix = extract_tfidf_features(df)
    # df['Tfidf_features'] = tfidf_matrix.tolist()

    # # Load Word2Vec model
    # word2vec_model = load_word2vec_model()

    # # Apply Word Embeddings
    # word_embeddings = extract_word_embeddings(df, word2vec_model)
    # df['Word_embeddings'] = word_embeddings

    # # Apply BERT Embeddings
    # bert_embeddings = extract_bert_embeddings(df)
    # df['Bert_embeddings'] = bert_embeddings

    return df

# Example usage:
data = {
    'Title': ['Apple announces new iPhone launch', 'Tesla stock surges after earnings report'],
    'Text': ['Apple unveiled its latest iPhone model at the event...', 'Tesla reported better-than-expected earnings...']
}
df = pd.DataFrame(data)

df = apply_all_techniques(df)


In [ ]:
new_tdf = apply_all_techniques(tdf)

In [ ]:
new_tdf

,links,Title,Text,PublishDate,Summary,Change,Bow_features
6,https://www.cnbc.com/2024/03/18/stocks-making-...,Stocks making the biggest moves premarket: Goo...,Check out the companies making headlines befor...,2024-03-18,Google — Alphabet Class A shares were trading ...,-0.889999,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,https://www.cnbc.com/2024/03/12/gm-hires-ex-te...,"GM hires ex-Tesla, Google exec to replace reti...",General Motors Executive Vice President Global...,2024-03-12,DETROIT – General Motors' long-time head of ma...,1.369995,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ..."
7,https://www.cnbc.com/2024/03/04/google-backed-...,"Anthropic, backed by Amazon and Google, debuts...","Anthropic on Monday debuted Claude 3, a suite ...",2024-03-04,"Anthropic on Monday debuted Claude 3, a suite ...",-2.339996,"[1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
13,https://www.cnbc.com/2024/02/28/most-searched-...,2 of the top 10 must-see travel experiences in...,Luxury travel company Kuoni recently released ...,2024-02-28,Luxury travel company Kuoni recently released ...,-1.670013,"[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, ..."
5,https://www.cnbc.com/2024/02/28/google-hit-wit...,Google hit with $2.3 billion lawsuit by Axel S...,The Google corporate logo hangs outside the Go...,2024-02-28,The Google corporate logo hangs outside the Go...,-1.670013,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,https://www.cnbc.com/2024/02/28/google-ceo-tel...,Google CEO tells employees Gemini AI blunder ‘...,"In a memo Tuesday evening, Google CEO Sundar P...",2024-02-28,"In a memo Tuesday evening, Google CEO Sundar P...",-1.670013,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,https://www.cnbc.com/2024/02/26/google-expects...,Google expects its more advanced large languag...,Google is bullish on the prospect of its more ...,2024-02-26,Google is bullish on the prospect of its more ...,-4.699997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,https://www.cnbc.com/2024/02/26/highmark-healt...,Highmark Health says it's combining tech from ...,Highmark Health announced Monday that it's int...,2024-02-26,Highmark Health announced Monday that it's int...,-4.699997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
10,https://www.cnbc.com/2024/02/26/googles-gemini...,Google to relaunch Gemini AI picture generator...,After pulling its artificial intelligence imag...,2024-02-26,Google introduced the image generator earlier ...,-4.699997,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,https://www.cnbc.com/2024/02/24/google-says-mi...,Google says Microsoft offered to sell Bing to ...,"Apple CEO Tim Cook, left, and Eddy Cue, Apple'...",2024-02-24,Microsoft offered to sell its Bing search engi...,-4.699997,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df = new_tdf

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming df is your DataFrame containing the article data with 'Bow_features' as Bag-of-Words features
X = df['Bow_features']
y = df['Change']  # Change represents the continuous stock price change

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SVR model
svr_model = SVR(kernel='linear')

# Train the model
svr_model.fit(X_train.tolist(), y_train)

# Predict on test data
y_pred = svr_model.predict(X_test.tolist())

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 4.5173068015428255


In [ ]:
for i in range (len(y_pred)):
  print(list(y_pred)[i],list(y_test)[i])

-1.0161986576454543 -2.4799957275390625
-1.4157449224536245 -1.8100128173828125
-0.3849635039907271 -0.57000732421875
-1.2100154794518625 1.2299957275390625
-1.0484038520282148 1.8499908447265625
-0.8358385976166429 -0.220001220703125
-0.5710431296632862 -1.670013427734375
-1.232104889811746 -0.79998779296875
0.3742777649038531 -4.6999969482421875
-0.6879997740097361 -1.670013427734375


In [ ]:
tdf.columns

Index(['links', 'Title', 'Text', 'PublishDate', 'Summary'], dtype='object')

In [ ]:
import yfinance as yf
import pandas as pd

def price_change(symbol, date, limit):
    if(limit<=0):
      return None

    stock_data = yf.download(symbol, start=date, end=(pd.to_datetime(date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d'))

    if not stock_data.empty:
      return stock_data['Close'].iloc[0] - stock_data['Open'].iloc[0]
    else:
      next_day = (pd.to_datetime(date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
      return price_change(symbol, next_day, limit -1)



def add_stock_change_column(df, symbol, publish_date_column):

    changes = []
    for publish_date in df[publish_date_column]:
        change = price_change(symbol, publish_date,limit = 10)
        changes.append(change)

    df['Change'] = changes

    return df

data = {
    'PublishDate': ['2022-06-15', '2022-06-16', '2022-06-17'],
    # Add other columns of your DataFrame
}
df = pd.DataFrame(data)

symbol = 'GOOG'  # Example symbol for Apple Inc.
publish_date_column = 'PublishDate'  # Column name containing publish dates

df = add_stock_change_column(tdf, symbol, publish_date_column)
# print(get_stock_price(symbol,'2024-03-18', limit=10))
# df


In [ ]:
import requests

company='Microsoft'
batch_size = 10

url = f"https://api.queryly.com/cnbc/json.aspx?queryly_key=31a35d40a9a64ab3&query={company}&endindex=40&batchsize={batch_size}&callback=&showfaceted=false&timezoneoffset=-330&facetedfields=formats&facetedkey=formats%7C&facetedvalue=!Press%20Release%7C&additionalindexes=4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # Analyze the data here
    # Example:
    print(len(data['results']))
    for i in data['results']:
      if i['cn:type']!='cnbcvideo':
        print(i['url'])
else:
    print("Error:", response.status_code)


10
https://www.cnbc.com/2024/02/24/google-says-microsoft-offered-to-sell-bing-to-apple-in-2018.html
https://www.cnbc.com/2024/02/28/microsoft-could-be-sitting-on-a-50-billion-opportunity-barclays-says.html
https://www.cnbc.com/2024/02/26/microsoft-invests-in-europes-mistral-ai-to-expand-beyond-openai.html
https://www.cnbc.com/2024/03/01/jim-cramers-friday-rapid-fire-dell-nycb-keurig-dr-pepper-zscaler-microsoft.html
https://www.cnbc.com/2024/03/15/fridays-top-stocks-to-buy-like-nvidia.html
https://www.cnbc.com/2024/03/14/10-things-to-watch-in-the-market-including-inflation-nvda-avgo-msft.html
https://www.cnbc.com/2024/03/13/stocks-to-buy-wednesday-like-nvidia.html
https://www.cnbc.com/2024/02/15/microsoft-will-bring-four-xbox-games-to-other-companies-consoles.html
https://www.cnbc.com/2024/02/13/apple-imessage-microsoft-bing-wont-count-as-gatekeepers-under-eu-dma.html


In [ ]:
Companies = ""

In [ ]:
import requests

company='Microsoft'
batch_size = 10

url = f"https://api.queryly.com/cnbc/json.aspx?queryly_key=31a35d40a9a64ab3&query={company}&endindex=40&batchsize={batch_size}&callback=&showfaceted=false&timezoneoffset=-330&facetedfields=formats&facetedkey=formats%7C&facetedvalue=!Press%20Release%7C&additionalindexes=4cd6f71fbf22424d,937d600b0d0d4e23,3bfbe40caee7443e,626fdfcd96444f28"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    # Analyze the data here
    # Example:
    print(len(data['results']))
    for i in data['results']:
      if i['cn:type']!='cnbcvideo':
        print(i['url'])
else:
    print("Error:", response.status_code)


In [ ]:
import yfinance as yf
import pandas as pd

def download_sp500_data(start_date, end_date):
    # Download data for S&P 500 companies
    sp500 = yf.Tickers('^GSPC')

    # Get ticker symbols
    tickers = sp500.tickers

    # Initialize a DataFrame to store historical data
    sp500_data = pd.DataFrame()

    # Retrieve historical data for each ticker
    for ticker in tickers:
        try:
            # Download historical data
            data = ticker.history(start=start_date, end=end_date)

            # Add ticker symbol as a column
            data['Ticker'] = ticker.ticker

            # Concatenate data to the main DataFrame
            sp500_data = pd.concat([sp500_data, data])
        except:
            pass

    return sp500_data

# Define start and end dates
start_date = '2023-01-01'
end_date = '2024-12-31'

# Download S&P 500 data from 2023 to 2024
sp500_data = download_sp500_data(start_date, end_date)

# Save the data to a CSV file
sp500_data.to_csv('sp500_data_2023_2024.csv')

# Display the first few rows of the data
print(sp500_data.head())


Empty DataFrame
Columns: []
Index: []
